In [1]:

# coding: utf-8

# In[141]:


import numpy as np
from keras.models import Model, load_model
from keras.layers import BatchNormalization,Convolution2D,SeparableConv2D,concatenate,Conv2DTranspose, Input, Add, UpSampling2D, Activation, merge, MaxPooling2D, Deconvolution2D, Reshape, Permute, Dropout
from keras.optimizers import SGD, Adam, Adadelta
from scipy.misc import imresize, imsave, imread
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau
from medpy.metric import dc, precision, recall
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator


def dice_coef(y_true, y_pred):
    
    #y_pred = K.round(y_pred)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return 1. -dice_coef(y_true, y_pred)


##load data and corresponding ground truth

train_X=np.load('train_cutimages_conall.npy' )
train_y=np.load('train_cutground.npy')
test_X=np.load('test1_cutimages_conall.npy')
test_y=np.load('test1_cutgrader3.npy')

#print (train_X.shape)
#print (train_y.shape)
#print (test_X.shape)
#print (test_y.shape)


# In[197]:


train_X = np.reshape(train_X, (-1,256,512,1))
train_y = np.reshape(train_y*1, (-1,256,512,1))
test_X = np.reshape(test_X, (-1,256,512,1))
test_y = np.reshape(test_y*1, (-1,256,512,1))

train_X = train_X.astype('float32')
train_y = train_y.astype('float32')
test_X = test_X.astype('float32')
test_y = test_y.astype('float32')

#print train_X.shape
#print train_y.shape
#print test_X.shape
#print test_y.shape


# In[198]:

### for data centering and normalization

mean = np.mean(train_X)
std = np.std(train_X)

#print mean, std

train_X -= mean
train_X /= std
test_X -= mean
test_X /= std


# In[199]:


#Test1 Data Creation volume-wise ###

val_x = []
val_gt = []
val_x.append(test_X[0:128])
val_gt.append(test_y[0:128])

val_x.append(test_X[128:256])
val_gt.append(test_y[128:256])

val_x.append(test_X[256:261])
val_gt.append(test_y[256:261])

val_x.append(test_X[261:266])
val_gt.append(test_y[261:266])

val_x.append(test_X[266:315])
val_gt.append(test_y[266:315])

val_x.append(test_X[315:364])
val_gt.append(test_y[315:364])

val_x.append(test_X[364:492])
val_gt.append(test_y[364:492])

val_x.append(test_X[492:620])
val_gt.append(test_y[492:620])


val_x = np.array(val_x)
val_gt = np.array(val_gt)






# In[176]:


datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, horizontal_flip=True, 
                             width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,fill_mode='constant')

datagen.fit(train_X)


# In[190]:


def create_model_girish(learn = 3e-4,wt='he_uniform',act='relu'): 

    inputs = Input((256, 512,1)) 
    conv1 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(inputs) 
    conv1 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(conv1)
    conv1 = BatchNormalization()(conv1)   
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) 

    conv2 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(pool1)
    conv2 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(conv2)    
    conv2 = BatchNormalization()(conv2) 
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) 

    conv3 = Convolution2D(128, 3, 3, activation=act, border_mode='same',init=wt)(pool2) 
    conv3 = Convolution2D(128, 3, 3, activation=act, border_mode='same',init=wt)(conv3) 
    conv3 = BatchNormalization()(conv3) 
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3) 

    conv4 = Convolution2D(256, 3, 3, activation=act, border_mode='same',init=wt)(pool3) 
    conv4 = Convolution2D(256, 3, 3, activation=act, border_mode='same',init=wt)(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4) 

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same',init=wt)(pool4) 
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same',init=wt)(conv5) 
    conv5 = BatchNormalization()(conv5) 
   

    input7 = UpSampling2D(size=(2, 2))(conv5) 
    input7 = Convolution2D(256,2,2,border_mode='same')(input7) 
    
    up7 = merge([input7, conv4], mode='concat') 
    conv7 = Convolution2D(256, 3, 3, activation='relu', border_mode='same',init='he_normal')(up7) 
    conv7 = BatchNormalization()(conv7) 
    conv7 = Convolution2D(256, 3, 3, activation='relu', border_mode='same',init='he_normal')(conv7) 
    conv7 = BatchNormalization()(conv7) 

    input8 = UpSampling2D(size=(2, 2))(conv7) 
    input8 = Convolution2D(128,2,2,border_mode='same')(input8) 
    up8 = merge([input8, conv3], mode='concat') 
    conv8 = Convolution2D(128, 3, 3, activation=act, border_mode='same',init=wt)(up8) 
    conv8 = BatchNormalization()(conv8) 
    conv8 = Convolution2D(128, 3, 3, activation=act, border_mode='same',init=wt)(conv8) 
    conv8 = BatchNormalization()(conv8) 

    input9 = UpSampling2D(size=(2, 2))(conv8) 
    input9 = Convolution2D(64,2,2,border_mode='same')(input9) 
  
    up9 = merge([input9, conv2], mode='concat') 
    conv9 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(up9) 
    conv9 = BatchNormalization()(conv9) 
    conv9 = Convolution2D(64, 3, 3, activation=act, border_mode='same',init=wt)(conv9) 
    conv9 = BatchNormalization()(conv9) 

    input10 = UpSampling2D(size=(2, 2))(conv9) 
    input10 = Convolution2D(32,2,2,border_mode='same')(input10) 
    
    up10 = merge([input10, conv1], mode='concat') 
    conv10 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(up10)
    conv10 = BatchNormalization()(conv10) 
    conv10 = Convolution2D(32, 3, 3, activation=act, border_mode='same',init=wt)(conv10) 
    conv10 = BatchNormalization()(conv10) 

    conv12 = Convolution2D(1, 1, 1,border_mode='same')(conv10) 

    conv13 = Activation('sigmoid')(conv12) 

    model = Model(input=inputs, output=conv13) 

    model.compile(Adam(lr=learn), loss='binary_crossentropy', metrics=[dice_coef,'accuracy']) 

    return model 


# In[191]:


K.clear_session()
learn = 3e-4 ## learning rate
wt='he_normal' ## weight initializer
act='relu'     ## activation function used
model = create_model_girish(learn,wt,act)
model.summary()

Using TensorFlow backend.
/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:134: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal", activation="relu")`
/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:135: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal", activation="relu")`
/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:139: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal", activation="relu")`
/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:140: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal", activation="relu")`
/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:144: UserWa

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 512, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 512, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 512, 32) 128         conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_poolin

/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:198: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`


In [2]:
### load best weights
model.load_weights('./best_weights.h5')

In [3]:
###testset####
total=0
for j in xrange(8):
    val_pred = model.predict(val_x[j], batch_size=1)
    val_result = np.zeros(val_pred.shape)
    val_result[val_pred>0.5] = 1
    dice = dc(val_result,val_gt[j])
    print "Volume "+str(j)+": "+str(dice)
    total+= dice
mean = total/8.0
print "Average dc: "+str(mean)

Volume 0: 0.0
Volume 1: 0.805500317326
Volume 2: 0.76466378496
Volume 3: 0.828125687436
Volume 4: 0.689782764383
Volume 5: 0.733624454148
Volume 6: 0.585087590166
Volume 7: 0.792256364604
Average dc: 0.649880120378


In [6]:
### test2 data split-up volume-wise ###

val_x = []
val_gt = []
val_x.append(test_X[0:128])
val_gt.append(test_y[0:128])

val_x.append(test_X[128:256])
val_gt.append(test_y[128:256])

val_x.append(test_X[256:261])
val_gt.append(test_y[256:261])

val_x.append(test_X[261:268])
val_gt.append(test_y[261:268])

val_x.append(test_X[268:275])
val_gt.append(test_y[268:275])

val_x.append(test_X[275:282])
val_gt.append(test_y[275:282])

val_x.append(test_X[282:289])
val_gt.append(test_y[282:289])


val_x = np.array(val_x)
val_gt = np.array(val_gt)


In [7]:
##test2###
total=0
for j in xrange(7):
        val_pred = model.predict(val_x[j], batch_size=4)
        val_result = np.zeros(val_pred.shape)
        val_result[val_pred>0.5] = 1  ##threshold used 0.5
        dice = dc(val_result,val_gt[j])
        print "Volume "+str(j)+": "+str(dice)
        total+= dice
mean = total/7.0
print "Average dc1: "+str(mean)


Volume 0: 0.779922440916
Volume 1: 0.848387296811
Volume 2: 0.839023858978
Volume 3: 0.657883797525
Volume 4: 0.865887973441
Volume 5: 0.834454230802
Volume 6: 0.914071175506
Average dc1: 0.819947253426
